In [2]:
import sympy as sp
import numpy as np
import matplotlib as plt

# https://docs.sympy.org/latest/index.html

In [95]:
#initialize symbols and time functions
t, x_0, y_0, v_0, Theta, g, m =sp.symbols(r't x_0 y_0 v_0 \Theta g m') 
g=sp.Matrix([[0],[g]])
x =sp.Function('x')(t)
y =sp.Function('y')(t)

""" x_dot = sp.Derivative(x,t)
x_ddot = x_dot.diff(t)
y_dot = sp.Derivative(y,t)
y_ddot = y_dot.diff(t) """

position = sp.Matrix([[x],[y]])
velocity = position.diff(t)
acceleration = velocity.diff(t)
motion_eqt = sp.Eq(m*acceleration,m*g)
motion_eqt


Eq(Matrix([
[m*Derivative(x(t), (t, 2))],
[m*Derivative(y(t), (t, 2))]]), Matrix([
[  0],
[g*m]]))

In [134]:
ode = m*acceleration-m*g
ode_flattened = ode.T.tolist()[0]
x_sol, y_sol=sp.dsolve(ode_flattened,[x,y])
x_sol, y_sol


(Eq(x(t), C1 + C2*t), Eq(y(t), C3 + C4*t + g*t**2/2))